In [98]:
# Import dependencies

import numpy as np
from random import seed
from random import random

In [99]:
# Initialise network with weights and biases for each layer

def initialise_network(n_inputs, n_hidden, n_outputs):
    network = []
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)] # plus 1 because of bias term
    network.append(output_layer)
    return network


In [100]:
# Forward propagation 

# Calculate weighted sum for an input 

def activate(weights, inputs):
    z = weights[-1]   # Assume the bias is the last in the list of weights
    for i in range(len(weights)-1):
        z += weights[i]*inputs[i]
    return z

# Put weighted sum through sigmoid activation function 

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

# Forward propagate input to a network output

def forward_propagate(network, row):
    inputs = row 
    for layer in network:
        new_inputs = []
        for neuron in layer:
            z = activate(neuron['weights'], inputs)
            neuron['output'] = sigmoid(z)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs


In [109]:
# Make a prediction with the network

# Take the argmax of the output of forward propagation, to select the class value with the largest probability 

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

# Print predictions 

for row in training_set:
    prediction = predict(network, row)
    print('Expected = %d, Got = %d' % (row[-1], prediction))

Expected = 0, Got = 0
Expected = 0, Got = 0
Expected = 0, Got = 0
Expected = 0, Got = 0
Expected = 0, Got = 0
Expected = 1, Got = 1
Expected = 1, Got = 1
Expected = 1, Got = 1
Expected = 1, Got = 1
Expected = 1, Got = 1


In [104]:
# Backpropagate error 

# Calculate sigmoid derivative of a neuron output

def sigmoid_prime(z):
    return z * (1.0 - z)

# Error in output layer = (expected - output) * sigmoid_prime(output)
# Error in hidden layer = (weight_k * error_j) * sigmoid_prime(output)

# Backpropagate error and store in neurons 

def backpropagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
            
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * sigmoid_prime(neuron['output'])
            

In [105]:
# Train network using stochastic gradient descent 

# Update weights

# new weight = weight + learning_rate * error * input

def update_weights(network, row, learning_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += learning_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += learning_rate * neuron['delta']   # The bias weight
            

# Loop for a fixed number of epochs; within each epoch, update the network for each row in the training dataset. This is called online learning.

# Train network for a fixed number of epochs

def train_network(network, training_set, learning_rate, n_epochs, n_outputs):
    for epoch in range(n_epochs):
        error_sum = 0
        for row in training_set:
            output = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]    # Transform binary class values in training data into a one hot encoding
            expected[row[-1]] = 1
            error_sum += sum([(expected[i] - output[i])**2 for i in range(len(expected))])
            backpropagate_error(network, expected)
            update_weights(network, row, learning_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, learning_rate, error_sum))
        

In [110]:
seed(1)

training_set = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

n_inputs = len(training_set[0]) - 1
n_outputs = len(set([row[-1] for row in training_set]))
network = initialise_network(n_inputs, 2, n_outputs)


train_network(network, training_set, 0.5, 20, n_outputs)
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': 0.0026279652850863837}]
[{'weights